### Mejora de Consultas – Técnicas de Expansión de Consultas

En un pipeline RAG, la calidad de la consulta enviada al recuperador determina qué tan bueno es el contexto recuperado — y por lo tanto, qué tan precisa será la respuesta final del LLM.

Ahí es donde entra la **Expansión / Mejora de Consultas**.

#### 🎯 ¿Qué es la Mejora de Consultas?
La mejora de consultas se refiere a técnicas utilizadas para mejorar o reformular la consulta del usuario para recuperar documentos mejores y más relevantes de la base de conocimiento.

Es especialmente útil cuando:

- La consulta original es corta, ambigua o está poco especificada
- Quieres ampliar el alcance para capturar sinónimos, frases relacionadas o variantes ortográficas
- Necesitas agregar contexto técnico o terminología específica del dominio

**Ventajas de la Expansión de Consultas:**
- ✅ Mejora la recuperación de documentos relevantes
- ✅ Captura variaciones semánticas de la consulta original
- ✅ Reduce falsos negativos (documentos relevantes no encontrados)
- ✅ Enriquece consultas vagas con contexto adicional

In [1]:
# Importación de librerías necesarias para el pipeline RAG con expansión de consultas

# TextLoader: Para cargar archivos de texto plano
from langchain.document_loaders import TextLoader

# RecursiveCharacterTextSplitter: Para dividir texto en fragmentos (chunks) manejables
from langchain.text_splitter import RecursiveCharacterTextSplitter

# HuggingFaceEmbeddings: Para generar embeddings vectoriales usando modelos de HuggingFace
from langchain_huggingface import HuggingFaceEmbeddings

# FAISS: Biblioteca de Facebook para búsqueda de similitud vectorial eficiente
from langchain_community.vectorstores import FAISS

# init_chat_model: Para inicializar modelos de chat de diferentes proveedores
from langchain.chat_models import init_chat_model

# PromptTemplate: Para crear plantillas de prompts con variables dinámicas
from langchain.prompts import PromptTemplate

# create_stuff_documents_chain: Crea una cadena que inserta documentos en un prompt
from langchain.chains.combine_documents import create_stuff_documents_chain

# create_retrieval_chain: Combina recuperación de documentos con generación de respuestas
from langchain.chains.retrieval import create_retrieval_chain

# StrOutputParser: Para convertir la salida del LLM a string
from langchain_core.output_parsers import StrOutputParser

# RunnableMap: Permite ejecutar múltiples operaciones en paralelo y combinar resultados
from langchain_core.runnables import RunnableMap

c:\Udemy\RAGBootcamp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Paso 1: Cargar y dividir el dataset

# Cargar el archivo de texto que contiene información sobre LangChain y CrewAI
loader = TextLoader("langchain_crewai_dataset.txt")

# load() devuelve una lista de documentos con todo el contenido del archivo
raw_docs = loader.load()

# Crear un splitter para dividir el texto en fragmentos más pequeños
# chunk_size=300: Cada fragmento tendrá máximo 300 caracteres
# chunk_overlap=50: Habrá una superposición de 50 caracteres entre fragmentos consecutivos
# La superposición ayuda a mantener el contexto entre fragmentos
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

# Dividir los documentos crudos en fragmentos (chunks) manejables
chunks = splitter.split_documents(raw_docs)

In [3]:
# Mostrar los fragmentos generados
# Esto nos permite ver cómo se dividió el documento
chunks

[Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain es un framework de cÃ³digo abierto diseÃ±ado para desarrollar aplicaciones basadas en grandes modelos de lenguaje (LLM). Simplifica el proceso de creaciÃ³n, gestiÃ³n y escalado de cadenas de pensamiento complejas al abstraer la gestiÃ³n de indicaciones, la recuperaciÃ³n, la memoria y la'),
 Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='indicaciones, la recuperaciÃ³n, la memoria y la orquestaciÃ³n de agentes. Los desarrolladores pueden usar LangChain para crear canales de extremo a extremo que conectan los LLM con herramientas, API, bases de datos vectoriales y otras fuentes de conocimiento. (v1)'),
 Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='La base de LangChain es el concepto de cadenas, que son secuencias de llamadas a LLM y otras herramientas. Las cadenas pueden ser simples, como una sola indicaciÃ³n enviada a un LLM, o complejas, con 

In [4]:
# Paso 2: Crear el almacén vectorial (Vector Store)

# Inicializar el modelo de embeddings de HuggingFace
# all-MiniLM-L6-v2 es un modelo compacto y eficiente que genera vectores de 384 dimensiones
# Este modelo es ideal para búsqueda semántica en español e inglés
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Crear el almacén vectorial FAISS a partir de los fragmentos de documentos
# FAISS convierte cada fragmento en un vector numérico para búsqueda de similitud eficiente
vectorstore = FAISS.from_documents(chunks, embedding_model)

# Paso 3: Crear el recuperador con MMR (Maximal Marginal Relevance)
# MMR balancea relevancia y diversidad para evitar documentos redundantes
retriever = vectorstore.as_retriever(
    search_type="mmr",  # Usar MMR en lugar de similitud simple
    search_kwargs={"k": 5}  # Recuperar los 5 documentos más relevantes y diversos
)

# Mostrar la configuración del recuperador
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021169E9F440>, search_type='mmr', search_kwargs={'k': 5})

In [5]:
# Paso 4: Configurar el LLM (Large Language Model) y variables de entorno

# Importar módulos para manejo de variables de entorno
import os
from dotenv import load_dotenv

# Cargar las variables de entorno desde el archivo .env
# Este archivo contiene las API keys de forma segura
load_dotenv()

# Establecer la API key de OpenAI desde las variables de entorno
# Esta clave es necesaria para usar los modelos de OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Inicializar el LLM usando el modelo o4-mini de OpenAI
# Este modelo es eficiente y económico para tareas de expansión de consultas
llm = init_chat_model("openai:o4-mini")

# Mostrar la configuración del LLM
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002116CF1E900>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002116D2361B0>, root_client=<openai.OpenAI object at 0x00000211676BA000>, root_async_client=<openai.AsyncOpenAI object at 0x000002116BCBAA50>, model_name='o4-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [6]:
# Paso 5: Crear la cadena de expansión de consultas

# Esta plantilla de prompt instruye al LLM para expandir la consulta del usuario
# El objetivo es agregar sinónimos, términos técnicos y contexto útil
query_expansion_prompt = PromptTemplate.from_template("""
Eres un asistente útil. Expande la siguiente consulta para mejorar la recuperación de documentos agregando sinónimos relevantes, términos técnicos y contexto útil.

Consulta original: "{query}"

Consulta expandida:
""")

# Crear la cadena de expansión de consultas
# Esta cadena conecta: prompt → LLM → parser de salida
# El operador | (pipe) encadena los componentes secuencialmente
query_expansion_chain = query_expansion_prompt | llm | StrOutputParser()

# Mostrar la estructura de la cadena de expansión
query_expansion_chain

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='\nEres un asistente útil. Expande la siguiente consulta para mejorar la recuperación de documentos agregando sinónimos relevantes, términos técnicos y contexto útil.\n\nConsulta original: "{query}"\n\nConsulta expandida:\n')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002116CF1E900>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002116D2361B0>, root_client=<openai.OpenAI object at 0x00000211676BA000>, root_async_client=<openai.AsyncOpenAI object at 0x000002116BCBAA50>, model_name='o4-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
| StrOutputParser()

In [7]:
# Probar la cadena de expansión de consultas
# Enviamos una consulta simple sobre "Langchain memory" para ver cómo la expande el LLM
# El LLM agregará términos técnicos, sinónimos y variaciones útiles
query_expansion_chain.invoke({"query": "Langchain memory"})

'Aquí tienes una posible versión de la consulta enriquecida, que incorpora sinónimos, términos técnicos y contexto para abarcar diversos aspectos de “LangChain memory”:\n\n("LangChain memory"  \n OR "LangChain memory management"  \n OR "LangChain memory module"  \n OR "LangChain converse memory"  \n OR "LangChain session memory"  \n OR "LangChain persistent memory"  \n OR "LangChain state management"  \n OR "LangChain context storage"  \n OR "LangChain cache"  \n OR "LangChain long-term memory"  \n OR "LangChain short-term memory"  \n OR "LangChain external memory"  \n OR "LangChain vector store memory"  \n OR "LangChain retrieval-augmented memory"  \n OR "LangChain RAG memory"  \n OR "LangChain embedding cache"  \n OR "stateful chatbots"  \n OR "conversational AI memory"  \n OR "context window management"  \n OR "contextual retrieval"  \n OR "memory classes in LangChain"  \n OR "gestión de memoria LangChain"  \n OR "módulo de memoria LangChain"  \n OR "almacenamiento de contexto"  \n 

In [8]:
# Paso 6: Crear el prompt para generar respuestas

# Esta plantilla define cómo el LLM usará el contexto recuperado para responder
answer_prompt = PromptTemplate.from_template("""
Responde la pregunta basándote en el contexto a continuación.

Contexto:
{context}

Pregunta: {input}
""")

# Crear la cadena de documentos que combina el LLM con el prompt de respuesta
# Esta cadena toma documentos recuperados y los inserta en el prompt
# "stuff" significa que todos los documentos se insertan directamente en el prompt
document_chain = create_stuff_documents_chain(llm=llm, prompt=answer_prompt)

In [9]:
# Paso 7: Construir el pipeline RAG completo con expansión de consultas

# Este pipeline integra la expansión de consultas en el flujo RAG:
# 1. Recibe la consulta original del usuario
# 2. Expande la consulta usando el LLM (agregando sinónimos y contexto)
# 3. Usa la consulta expandida para recuperar documentos relevantes
# 4. Genera la respuesta final usando los documentos recuperados

rag_pipeline = (
    # RunnableMap ejecuta múltiples operaciones en paralelo
    RunnableMap({
        # Mantener la consulta original para la respuesta final
        "input": lambda x: x["input"],
        
        # Flujo de recuperación con expansión de consultas:
        # 1. Toma la consulta original (x["input"])
        # 2. La expande usando query_expansion_chain
        # 3. Usa la consulta expandida para invocar el retriever
        # 4. Devuelve los documentos recuperados como contexto
        "context": lambda x: retriever.invoke(
            query_expansion_chain.invoke({"query": x["input"]})
        )
    })
    # El resultado del RunnableMap se pasa a document_chain para generar la respuesta
    | document_chain
)

In [10]:
# Paso 8: Ejecutar una consulta de ejemplo sobre tipos de memoria en LangChain

# Definir la consulta del usuario
query = {"input": "¿Qué tipos de memoria soporta LangChain?"}

# Primero, mostrar cómo se expande la consulta
# Esto nos permite ver qué términos adicionales agregó el LLM
print("🔍 Consulta Expandida:")
print(query_expansion_chain.invoke({"query": query["input"]}))
print("\n" + "="*80 + "\n")

# Invocar el pipeline RAG completo con la consulta
# El pipeline automáticamente:
# 1. Expandirá la consulta
# 2. Recuperará documentos relevantes
# 3. Generará una respuesta basada en el contexto
response = rag_pipeline.invoke(query)

# Mostrar la respuesta final del sistema RAG
print("✅ Respuesta:")
print(response)

🔍 Consulta Expandida:
Consulta ampliada:

“¿Cuáles son los distintos mecanismos y objetos de ‘Memory’ que soporta el framework LangChain para gestionar el contexto y el estado de las conversaciones con modelos de lenguaje? En concreto, me interesa conocer:

1. Tipos de memoria integrados en LangChain  
   - ConversationBufferMemory (caché de diálogo completo)  
   - ConversationSummaryMemory (resumen incremental)  
   - ConversationEntityMemory (extracción y seguimiento de entidades)  
   - VectorStoreRetrieverMemory o VectorStoreMemory (memoria basada en vectores)  
   - CombinedMemory (combinación de varios back-ends)  
   - LongTermMemory (memoria a largo plazo)

2. Sinónimos y términos relacionados  
   - State management, session state, chat history  
   - Context persistence, almacenamiento de contexto, retención de estado  
   - Embedding store, vector database, base de datos de vectores

3. Opciones de persistencia y back-ends compatibles  
   - Redis, FAISS, ChromaDB, Milvus  

In [11]:
# Paso 9: Ejecutar una segunda consulta sobre agentes de CrewAI

# Definir una consulta más corta y menos específica
# Esta consulta se beneficiará especialmente de la expansión
query = {"input": "¿Agentes CrewAI?"}

# Mostrar la consulta expandida para ver el enriquecimiento
print("🔍 Consulta Expandida:")
print(query_expansion_chain.invoke({"query": query["input"]}))
print("\n" + "="*80 + "\n")

# Invocar el pipeline RAG completo
# Observa cómo una consulta muy corta ("¿Agentes CrewAI?") se convierte en
# una búsqueda enriquecida con contexto técnico y sinónimos
response = rag_pipeline.invoke(query)

# Mostrar la respuesta generada
print("✅ Respuesta:")
print(response)

🔍 Consulta Expandida:
Consulta expandida:

(“CrewAI” OR “Crew AI” OR “Crew-AI” OR “plataforma CrewAI” OR “framework CrewAI”)  
AND  
(“agente inteligente” OR “agente conversacional” OR “agente autónomo” OR “bot de IA” OR “agente de inteligencia artificial”)  
AND  
(“inteligencia artificial” OR “IA” OR “machine learning” OR “aprendizaje automático” OR “NLP” OR “procesamiento de lenguaje natural” OR “LLM” OR “modelos de lenguaje” OR “transformers”)  
AND  
(“documentación técnica” OR “guía de uso” OR “guía de implementación” OR “SDK” OR “API” OR “librería” OR “versión” OR “release notes”)  
AND  
(“arquitectura” OR “orquestación de agentes” OR “diseño multiagente” OR “integración” OR “casos de uso” OR “patrones de diseño” OR “benchmark” OR “rendimiento” OR “escalabilidad” OR “deployment” OR “CI/CD”)  
AND  
(“automatización de tareas” OR “asistentes virtuales” OR “sistemas de diálogo” OR “chatbots” OR “workflow inteligente” OR “pipeline de IA”)  

Esta consulta combina sinónimos, términ

### 📊 Resumen: Beneficios de la Expansión de Consultas

**Antes de la expansión:**
- Consulta: "Langchain memory"
- Problema: Muy específica, puede perder documentos relevantes que usen terminología diferente

**Después de la expansión:**
- Consulta expandida incluye: "ConversationBufferMemory", "ConversationSummaryMemory", "memory management", "session context", etc.
- Resultado: Mejor cobertura de documentos relevantes

**Casos de uso ideales:**
- ✅ Consultas cortas o vagas
- ✅ Búsquedas técnicas que requieren terminología específica
- ✅ Cuando el usuario no conoce los términos exactos
- ✅ Mejorar recall (recuperación) sin sacrificar precisión

**Consideraciones:**
- ⚠️ Agrega latencia (~500ms-1s por la llamada al LLM)
- ⚠️ Puede sobre-expandir consultas ya muy específicas
- ⚠️ Requiere una API key y tiene costo asociado